# Q1: Simple CNN network for PASCAL multi-label classification (20 points)
Now let’s try to recognize some natural images. We provided some starter code for this task. The following steps will guide you through the process.


## 1.1 Setup the dataset
We start by modifying the code to read images from the PASCAL 2007 dataset. The important thing to note is that PASCAL can have multiple objects present in the same image. Hence, this is a multi-label classification problem, and will have to be tackled slightly differently.


First, download the data. `cd` to a location where you can store 0.5GB of images. Then run:
```
wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
tar -xf VOCtrainval_06-Nov-2007.tar

wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
tar -xf VOCtest_06-Nov-2007.tar
cd VOCdevkit/VOC2007/
```

## 1.2 Write a dataloader with data augmentation (5 pts)
**Dataloader** The first step is to write a [pytorch data loader](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html) which loads this PASCAL data. Browse the folders and files under `VOCdevkit` to understand the structure and labeling. Complete the functions `preload_anno` and `__getitem__` in `voc_dataset.py` according to the following instructions and the instructions in the code. More information about the dataset can be found [here](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/devkit_doc_07-Jun-2007.pdf). We will use data in ‘trainval’ for training and ‘test’ for testing.

- `preload_anno`: This function will be called when the dataloader is initialized. We will load the annotations under folder `Annotations`. Each .xml file in the `Annotations` folder corresponds to the image with the same name under `JPEGImages`. In this function, we need to load `label` and `weight` vectors for each image according to the .xml file. 
- The labels should be 0 by default. Assign 1 for each class label in the .xml file. For example, in 000001.xml, the label vector should have 1s at the class indices correspond to 'dog' and 'person'. The rest of the vector should be 0.
- The weights should be 1 by defatul. For each class label in the image, if 'difficult'=1 (which means it is ambiguous), we will assign 0 for the weight vector at this class index. This weight will be used when we calculate the test performance. We will not consider the ambiguous labels during testing.

- `__getitem__`: This function will be called when the dataloader is called during training. It takes as input the index, and returns a tuple - `(image, label, weight)`. You need to load the image from the `JPEGImages` folder and load the corresponding label and weight using `self.anno_list`.


**Data Augmentation** Modify `__getitem__` to randomly *augment* each datapoint using [TORCHVISION.TRANSFORMS](https://pytorch.org/vision/stable/transforms.html).  Make sure the data augmentation is only used for training data (based on self.split). Please describe what data augmentation you implement.

- Before any augmentation, resize all the images based on `self.size`.

* **Hint**: Since we are training a model from scratch on this small dataset, it is important to perform basic data augmentation to avoid overfitting. Add random crops and left-right flips when training, and do a center crop when testing, etc. As for natural images, another common practice is to subtract the mean values of RGB images from ImageNet dataset. The mean values for RGB images are: `[123.68, 116.78, 103.94]`. You may also rescale the images to `[−1, 1]`. There is no "correct" answer here! Feel free to search online about the data augmentation methods people usually use.

### DESCRIBE YOUR AUGMENTATION PIPELINE HERE**
**Train Augmentations:**
For training dataset, I used the following transformations for data augmentation.
- Resize - resized the image to the desired image input size
- Random Crop - random crop while preserving image size
- Random Horizontal Flip
- Normalize - performed normalization based on the mean RGB values and scaled it between 0 and 1 
<br>

**Test Augmentations:**
For training dataset, I used the following transformations for data augmentation.
- Resize
- Center Crop
- Normalize

## 1.3 Measure Performance (5 pts)
To evaluate the trained model, we will use a standard metric for multi-label evaluation - [mean average precision (mAP)](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). Please implement `eval_dataset_map` in `utils.py` - this function will evaluate a model's map score using a given dataset object. You will need to make predictions on the given dataset with the model and call `compute_ap` to get average precision.


Please describe how to compute AP for each class(not mAP).

**YOUR ANSWER HERE**

Precision and recall are used to measure performance of prediction models. 
Precision is the ratio of the number of positive class predictions to true and false positives. Recall is the ratio of number of positive class predictions to number of all true positives and false negatives in the dataset.

Average Precision or AP is the area under the curve in the precision vs recall plot. In a sorted list of propabilities of classification, infinitesimal steps are taken from probability of zero to probability of one, and the precision and recall at each interval is calculated. AP is a measure that combines recall and precision for ranked retrival results.

## 1.4 Let's Start Training! (5 pts)
Fill out the loss function for multi-label classification in `trainer.py` and start training. In this question, you will use the model that you finished in the previous question (with proper non-linearities).

Initialize a fresh model and optimizer. Then run your training code for 5 epochs and print the mAP on test set. The resulting mAP should be around 0.24. Make sure to tune the hyperparameters.

In [1]:
import torch
import trainer
from utils import ARGS
from simple_cnn import SimpleCNN
from voc_dataset import VOCDataset

# create hyperparameter argument class
# Use image size of 64x64 in Q1. We will use a default size of 224x224 for the rest of the questions.
args = ARGS(epochs=5, inp_size=64)
print(args)

args.batch_size = 64
args.device = cpu
args.epochs = 5
args.gamma = 0.7
args.inp_size = 64
args.log_every = 100
args.lr = 1.0
args.save_at_end = False
args.save_freq = -1
args.step_size = 1
args.test_batch_size = 1000
args.val_every = 100



In [2]:
args = ARGS(epochs=5, inp_size=64, lr = 1e-3, step_size=1, use_cuda = True, gamma = 0.8, batch_size = 32, test_batch_size = 64)
print(args)
# initializes the model
model = SimpleCNN(num_classes=len(VOCDataset.CLASS_NAMES), inp_size=64, c_dim=3)
# initializes Adam optimizer and simple StepLR scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)
# trains model using your training code and reports test map
test_ap, test_map = trainer.train(args, model, optimizer, scheduler)
print('test map:', test_map)

args.batch_size = 32
args.device = cuda
args.epochs = 5
args.gamma = 0.8
args.inp_size = 64
args.log_every = 100
args.lr = 0.001
args.save_at_end = False
args.save_freq = -1
args.step_size = 1
args.test_batch_size = 64
args.val_every = 100

Train Epoch: 0 [0 (0%)]	Loss: 0.694806
Train Epoch: 0 [100 (64%)]	Loss: 0.254486
Train Epoch: 1 [200 (27%)]	Loss: 0.232630
Train Epoch: 1 [300 (91%)]	Loss: 0.214726
Train Epoch: 2 [400 (55%)]	Loss: 0.192280
Train Epoch: 3 [500 (18%)]	Loss: 0.200821
Train Epoch: 3 [600 (82%)]	Loss: 0.178838
Train Epoch: 4 [700 (46%)]	Loss: 0.228114
test map: 0.2404556895123307


[TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard) is an awesome visualization tool. It was firstly integrated in [TensorFlow](https://www.tensorflow.org/). It can be used to visualize training losses, network weights and other parameters.

To use TensorBoard in Pytorch, there are two options: [TensorBoard in Pytorch](https://pytorch.org/docs/stable/tensorboard.html) (for Pytorch >= 1.1.0) or [TensorBoardX](https://github.com/lanpa/tensorboardX) - a third party library. Following these links to add code in `trainer.py` to visualize the testing MAP and training loss in Tensorboard. *You may have to reload the kernel for these changes to take effect.*

Show clear screenshots of the learning curves of testing MAP and training loss for 5 epochs (batch size=20, learning rate=0.001). Please evaluate your model to calculate the MAP on the testing dataset every 100 iterations.

In [2]:
args = ARGS(epochs=5, batch_size=20, lr=0.001, inp_size=64, gamma=0.8, step_size=2, use_cuda=True)
print(args)
model = SimpleCNN(num_classes=len(VOCDataset.CLASS_NAMES), inp_size=64, c_dim=3)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)
test_ap, test_map = trainer.train(args, model, optimizer, scheduler)
print('test map:', test_map)

args.batch_size = 20
args.device = cuda
args.epochs = 5
args.gamma = 0.8
args.inp_size = 64
args.log_every = 100
args.lr = 0.001
args.save_at_end = False
args.save_freq = -1
args.step_size = 2
args.test_batch_size = 1000
args.val_every = 100

Train Epoch: 0 [0 (0%)]	Loss: 0.703466
Train Epoch: 0 [100 (40%)]	Loss: 0.224520
Train Epoch: 0 [200 (80%)]	Loss: 0.260876
Train Epoch: 1 [300 (20%)]	Loss: 0.182323
Train Epoch: 1 [400 (59%)]	Loss: 0.194093
Train Epoch: 1 [500 (99%)]	Loss: 0.215749
Train Epoch: 2 [600 (39%)]	Loss: 0.164597
Train Epoch: 2 [700 (79%)]	Loss: 0.219819
Train Epoch: 3 [800 (19%)]	Loss: 0.205551
Train Epoch: 3 [900 (59%)]	Loss: 0.206481
Train Epoch: 3 [1000 (98%)]	Loss: 0.177016
Train Epoch: 4 [1100 (38%)]	Loss: 0.206823
Train Epoch: 4 [1200 (78%)]	Loss: 0.184720
test map: 0.28257098106488726


**INSERT YOUR TENSORBOARD SCREENSHOTS HERE**

***Loss for training***

<img src="vlr-hw1-images/q1-loss.png"/>


***mAP for testing*** 

<img src="vlr-hw1-images/q1-map.png"/>